In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
#from balm.config import BalmConfig, BalmMoEConfig, BalmExpertChoiceMoEConfig
from balm.data import load_dataset, DataCollator
# from balm.models import (
#     BalmForMaskedLM,
#     BalmModel,
#     BalmMoEForMaskedLM,
#     BalmExpertChoiceMoEForMaskedLM,
#     BalmHybridMoEForMaskedLM,
#     BalmMoERoPEForMaskedLM,
# )
from balm.tokenizer import Tokenizer
from balm.train import Trainer

In [3]:
from transformers import (
    EsmConfig,
    EsmTokenizer,
    EsmForProteinFolding,
)

In [4]:
# tokenizer = Tokenizer(vocab="./vocab.json")

In [5]:
tokenizer = EsmTokenizer.from_pretrained('facebook/esmfold_v1')

/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
data_files = {
    "train": '../protein-folding/data/processed/3A-cutoffs/below3A/20240501_below3A-human-clust100_train.parquet',
}
dataset = load_dataset("parquet", data_files=data_files)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 7: invalid start byte

In [7]:
from datasets import load_dataset

In [8]:
data_files = {
    "train": '../protein-folding/data/processed/3A-cutoffs/below3A/20240501_below3A-human-clust100_train.parquet',
}
dataset = load_dataset("parquet", data_files=data_files)

In [9]:
tokenized_dataset = dataset.map(
    lambda x: tokenizer(
        x["sequence"],
        add_special_tokens=False,
    ),
    remove_columns="sequence"
)

In [10]:
# collator = DataCollator(tokenizer=tokenizer)

In [11]:
from fold_utils.collator import FoldDataCollator

In [12]:
collator = FoldDataCollator()

In [ ]:
# model = BalmMoERoPEForMaskedLM(
#     embed_dim=320,
#     ffn_dim=320*4,
#     num_experts=4,
#     # num_shared_experts=0,
#     num_layers=6,
#     num_heads=20,
#     # alternate_sparsity=True,
#     # router_top_k=1,
#     expert_capacity=1.25*320/16,
#     expert_activation="swiglu",
#     router_z_loss_coef=0.01,
#     router_aux_loss_coef=0.01,
#     vocab_size=tokenizer.vocab_size,
# )
# model = BalmMoEForMaskedLM(config=config)

In [ ]:
# model.num_parameters

In [13]:
config = EsmConfig(is_folding_model=True, 
                   #esmfold_config=fold_config,
                   vocab_size=26, # required, needs to be base model size
                   vocab_list=tokenizer.all_tokens,
                   pad_token_id=21,
                   num_attention_heads=20,
                   num_hidden_layers=32,
                   hidden_size=960,
                   intermediate_size=3840,
                   max_len=320, 
                   max_position_embeddings=322,
                   attention_probs_dropout_prob=0.0,
                   esm_layer_norm_before=False,
                   hidden_dropout_prob=0.0,
                   layer_norm_eps=1e-5,
                   position_embedding_type="rotary",)

In [14]:
model = EsmForProteinFolding.from_pretrained('../protein-folding/models/paired-esm-model/', config=config)

Some weights of EsmForProteinFolding were not initialized from the model checkpoint at ../protein-folding/models/paired-esm-model/ and are newly initialized: ['af2_to_esm', 'distogram_head.bias', 'distogram_head.weight', 'embedding.weight', 'encoder.layer.0.attention.self.rotary_embeddings.inv_freq', 'encoder.layer.1.attention.self.rotary_embeddings.inv_freq', 'encoder.layer.10.attention.self.rotary_embeddings.inv_freq', 'encoder.layer.11.attention.self.rotary_embeddings.inv_freq', 'encoder.layer.12.attention.self.rotary_embeddings.inv_freq', 'encoder.layer.13.attention.self.rotary_embeddings.inv_freq', 'encoder.layer.14.attention.self.rotary_embeddings.inv_freq', 'encoder.layer.15.attention.self.rotary_embeddings.inv_freq', 'encoder.layer.16.attention.self.rotary_embeddings.inv_freq', 'encoder.layer.17.attention.self.rotary_embeddings.inv_freq', 'encoder.layer.18.attention.self.rotary_embeddings.inv_freq', 'encoder.layer.19.attention.self.rotary_embeddings.inv_freq', 'encoder.layer.2.

In [15]:
model_size = sum(p.numel() for p in model.parameters())
print(f"Model size: {model_size/1e9:.2f}B")

Model size: 1.05B


In [16]:
trainer = Trainer(
    model=model,
    data_collator=collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["train"],
    output_dir="./models/",
    epochs=1,
    logging_steps=5,
    eval_steps=100,
    warmup_steps=10,
    # save_steps=15,
    per_device_train_batch_size=1,
    # use_cpu=True,
    # use_wandb=True,
    wandb_project="test_wandb_logging",
    # wandb_entity="bryanbriney",
    run_name="save_test_001",
)

In [17]:
trainer.train()

Training:   0%|          | 0/1398 [00:00<?, ?step/s]

KeyError: 0